# Training and Evaluating models:

In [6]:
import pandas as pd

# Define Excel file path
excel_file = "../../Types_Cutting_Inserts.xlsx"
csv_file = "Wear_data.csv"
insert_type_list = ["RM121263NE-BB", "RM090955NE-AB", "RM090955NE-AC", "RM121279NE-CV", "RM121279NE-DF", "RM121279NE-CU", "SNC-44-170", "SNC-44-60KH04"]
# Read all sheets into a dictionary of DataFrames
sheets = pd.read_excel(excel_file, sheet_name=None)

# Create an empty list to store data
data_list = []

# Process each sheet
for insert_type, df in sheets.items():
    if insert_type in insert_type_list:
        df = df.iloc[:, :5] 
        # Ensure column names are correct
        df.columns = ["Insert_Name", "TOP", "LEFT", "RIGHT", "BOTTOM"]
        
        # Append to list
        data_list.append(df)

# Combine all sheets into one DataFrame
final_df = pd.concat(data_list, ignore_index=True)

# Save to CSV
final_df.to_csv(csv_file, index=False)
print(f"CSV file saved as {csv_file}")

CSV file saved as Wear_data.csv


## Training

In [ ]:
! python train.py \
    --model_name resnet18 \
    --ckpts_path ./ckpts \
    --data_path ../../dataset \
    --csv_path Wear_data.csv \
    --num_folds 5 \
    --val_fold 0 \
    --batch_size 128 \
    --val_batch_size 128 \
    --num_workers 0 \
    --size 224 \
    --lr 0.05 \
    --momentum 0 \
    --weight_decay 0 \
    --num_epochs 10 \
    --wandb_entity WANDB_ENTITY \
    --wandb_project WANDB_PROJECT \
    --loss_weight_class 2500.0 \
    --loss_weight_reg 1.0 \
    --split 0.9

: 

# Evaluation

In [2]:
! python test.py \
    --model_name resnet18 \
    --load_ckpt ./ckpts/multiTask_lr=0.1;epoch=500;loss_weight=2500.pth \
    --data_path ../../dataset \
    --csv_path Wear_data.csv \
    --num_workers 0 \
    --val_batch_size 32 \
    --split 0.9

Overall Report:
--------------------------------------------------
Classification Accuracy: 100.00%
Regression Accuracy Range (within 20.0 percentage or 50µm tolerance): 70.44%
Binary Regression Accuracy: (Threshold = 200µm): 89.70%
False Positive Rate (Not worn-out but detected): 3.98%
False Negative Rate (Worn-out but not detected): 6.32%
Mean Squared Error (MSE): 500.0585
Mean Absolute Error (MAE): 1.0870
Total Targets: 2612

Detailed Report:
--------------------------------------------------
Classification Report:
                precision    recall  f1-score   support

RM090955NE-AB       1.00      1.00      1.00         7
RM090955NE-AC       1.00      1.00      1.00        14
RM121263NE-BB       1.00      1.00      1.00        29
RM121279NE-CU       1.00      1.00      1.00        25
RM121279NE-CV       1.00      1.00      1.00        19
RM121279NE-DF       1.00      1.00      1.00        23
   SNC-44-170       1.00      1.00      1.00        19
SNC-44-60KH04       1.00      1.00


100%|██████████| 6/6 [00:06<00:00,  1.00s/it]
